In [1]:
import glob
import os
import pandas as pd
import numpy as np
import math
import scipy.stats as st

import qiime2 as q2

In [3]:
!qiime decontam hello --help

Usage: qiime decontam hello [OPTIONS]

  print 'hello world'

Outputs:
  --o-text ARTIFACT    
    Greeting                                                        [required]
Miscellaneous:
  --output-dir PATH    Output unspecified results to a directory
  --verbose / --quiet  Display verbose output to stdout and/or stderr during
                       execution of this action. Or silence output if
                       execution is successful (silence is golden).
  --examples           Show usage examples and exit.
  --citations          Show citations and exit.
  --help               Show this message and exit.


In [5]:
!make run

qiime decontam hello --o-text example_greeting --verbose
Saved Greeting to: example_greeting.qza


def prepare control batches (feature table, metadata, name of sample_type col, name of experimental samples in sample_type col (default to experimental, optional parameter name_of_experimental_sample = 'experimental'))
    create a list of unique values for each control type metadata col sample_type
    use filter samples to create a list of samples where sample_type = each unique value 
    return lists of sample ids associated with each sample_type except
    use filter features on returned lists keep only features present in at least 2 samples were sample_type = experimental
    return subsets of feature tables with features in 2 or more experimental samples named by sample_type
    
    look at my py docs
    
    from typing import List, Tuple
        List[biom.Table] are mutable

        Tuple[biom.Table, ...] immutable 
        
        ar_1, var2 = some_func_that_returns a two-tuple
        var_1 first value from the tuple 
base class - underlying functionality - don't use
concrete - childern of base classes - concrete implimentation 

https://stackoverflow.com/questions/13957829/how-to-use-raise-keyword-in-python/13957915#13957915

https://docs.pytest.org/en/6.2.x/goodpractices.html#test-discovery

q2 lint
BSD3 clause license
regex to check header against some standard for q2
require that q2 is in header 